In [22]:
# Dependencies
import warnings
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
from pydantic import BaseModel
import openai
import yfinance as yf


In [23]:
# Load environment variables from the .env file
load_dotenv()

# Fetch API keys and model name from the .env file
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

# Set API keys
openai.api_key = OPENAI_API_KEY

# Instantiate tools
news_tool = SerperDevTool(api_key = SERPER_API_KEY)
scrape_tool = ScrapeWebsiteTool()


In [24]:
# News Retrival Agent
news_retrival_agent = Agent(
    role = "News Retrival Agent",
    goal = "Fetch the latest news about the company",
    backstory = "You collect the most recent news articles elated to a company for sentiment analysis.",
    verbose = True,
    tools = [news_tool, scrape_tool]
)

In [25]:
# Sentiment Analysis Agent
sentiment_analysis_agent = Agent(
    role = "Sentiment Analysis Agent",
    goal = "Analyze the sentiment from recent news articles",
    backstory = "You analyze the sentiment of the news articles between -1 (very negitive) and 1 (very positive) and provide an aggregate sentiment score.",
    verbose = True,
    #allow_delegation = False
)

In [26]:
# Tasks
retrieve_news_task = Task(
    description = "Fetch recent news articles about {company_name}",
    expected_output = "A list of recent news articles.",
    agent = news_retrival_agent,
    #tools = [news_tool, scrape_tool]
)

sentiment_analysis_task = Task(
    description = "Analyze the sentiment of he news articles for {company_name}",
    expected_output = "An aggregate sentiment score for the articles",
    agent = sentiment_analysis_agent
)


In [27]:
# Define the Crew and assign Tasks
financial_analysis_crew = Crew(
    agents = [news_retrival_agent, sentiment_analysis_agent],
    tasks = [retrieve_news_task, sentiment_analysis_task],
    verbose = True
)

2024-10-24 15:22:46,700 - 140704492328576 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [28]:
# Inputs and Outputs
stock_ticker = input("Please enter the company ticker symbol: ").upper()
if stock_ticker:
    print(f"Stock ticker provided: {stock_ticker}")
    result = financial_analysis_crew.kickoff(inputs = {"company_name": stock_ticker})
    if result:
        sentiment_analysis_result = result.task_results("sentiment_analysis_task")
        aggregate_score = sentiment_analysis_result["aggregate_score"] if sentiment_analysis_result else 0
        print(f"Final Aggregate Sentiment Score: {aggregate_score:.2f}")
    else:
        print("No articles found.")
else:
    print("Invalid icker symbol.")

Stock ticker provided: TSLA
# Agent: News Retrival Agent
## Task: Fetch recent news articles about TSLA


# Agent: News Retrival Agent
## Thought: I need to search for the most recent news articles related to TSLA (Tesla, Inc.) to fulfill the task.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"recent news articles about TSLA\"}"
## Tool Output: 

Search results: Title: Tesla news - Today's latest updates
Link: https://www.cbsnews.com/tag/tesla/
Snippet: Tesla profits plunge as automaker grapples with slumping EV sales. Electric Vehicles Tesla cuts prices around the globe amid slowing demand for its EVs.
---
Title: News | Tesla
Link: https://www.tesla.com/blog
Snippet: We are launching the new Model 3 Performance: a highly differentiated performance trim that leverages Tesla's latest manufacturing and engineering capabilities.
---
Title: Tesla Motors Inc. - The New York Times
Link: https://www.nytimes.com/topic/company/tesla-motors-inc
Snippet: Elon Musk Unvei

AttributeError: 'CrewOutput' object has no attribute 'task_results'